<a href="https://colab.research.google.com/github/stefanbringuier/randomonium/blob/main/notebooks/AutoMorse.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Morse Autofit Quantum Chemistry Diatomic Binding Curves

🖊 **Author:** Stefan Bringuier  
📧 **Email:** [stefanbringuier@gmail.com](mailto:stefanbringuier@gmail.com)  
🔖 **License:** MIT  
🌍 **Website:** [stefanbringuier.info](https://stefanbringuier.info)  


This was just a quick test script/notebook where I calculate the binding curves for diatomic molecules and fit to a basic Morse potential and is captured as an [ASE Calculator](https://wiki.fysik.dtu.dk/ase/ase/calculators/calculators.html).

Approach is:

1. Pick your level-of-theory and basis set consistent with [PySCF](https://pyscf.org/) naming.
2. Generate binding curves. Inspect plots and parameter file.
3. Use ASE calculator if desired.

Not entirely sure how useful this is for MD simulations, but might be useful when needing interaction terms for one-off species and bulk/surfaces.


> 🚨 **verify the fits and parameters yourself!**  
> The automated fitting process may not always yield physically meaningful results.  
> Before using the fitted Morse potentials, **inspect the binding curve plots and the parameter files** (`morse_parameters.toml`).  
> If the fits look incorrect, adjust the fitting method or bounds as needed.


In [64]:
%%capture
! pip install -U ase
! pip install -U pyscf

In [67]:
import os
import re
import numpy as np
import toml
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement
from ase import Atoms
from ase.data import atomic_numbers
from ase.calculators.morse import MorsePotential
from pyscf import gto, scf, mp, cc
from scipy.optimize import minimize
from tqdm import tqdm

__author__ = "Stefan Bringuier <stefanbringuier@gmail.com>"
__version__ = "1.0.0"
__license__ = "MIT"

HARTREE_TO_EV = 27.2114

class AutoMorsePotential(MorsePotential):
    """
    ASE MorsePotential Calculator with automatic parameter fitting using PySCF.

    - Inherits from `MorsePotential` and dynamically updates parameters.
    - Uses quantum chemistry (PySCF) to fit Morse parameters for each diatomic pair.
    - Saves Morse parameters to a YAML/TOML file.
    """

    def __init__(self, molecule=None, qc_kwargs=None, opt_kwargs=None, **kwargs):
        self.pair_params = {}
        self.binding_curves = {}

        qc_defaults = {
            "method": "CCSD(T)",
            "basis": "aug-cc-pVQZ",
            "conv_tol": 1e-12,
            "conv_tol_grad": 1e-8,
            "max_cycle": 500,
        }
        self.qc_kwargs = {**qc_defaults, **(qc_kwargs or {})}
        self.opt_kwargs = {"maxiter": 1000, "ftol": 1e-6}
        self.opt_kwargs.update(opt_kwargs or {})

        print(
            f"Using level of theory: {self.qc_kwargs['method']} | Basis set: {self.qc_kwargs['basis']}"
        )

        if molecule:
            self.molecule = molecule
            elements = self._parse_molecule(molecule)
            self.auto_fit_all_pairs(elements)

        super().__init__(epsilon=1.0, rho0=1.0, r0=1.0, rcut1=1.9, rcut2=2.7, **kwargs)

    def _parse_molecule(self, molecule_spec):
        if isinstance(molecule_spec, str):
            elems = re.findall(r"[A-Z][a-z]?", molecule_spec.replace("-", ""))
        elif isinstance(molecule_spec, list):
            elems = molecule_spec
        else:
            raise ValueError("Unsupported molecule specification format.")
        return list(set(elems))

    def auto_fit_all_pairs(self, elements, rlow=0.35, rhigh=3.5, npoints=24):
        for pair in combinations_with_replacement(sorted(elements), 2):
            params, r_vals, E_bind = self.auto_fit_pair(pair, rlow, rhigh, npoints)
            self.pair_params["-".join(pair)] = params
            self.binding_curves[pair] = (r_vals, E_bind)

        self._save_parameters()

        self._plot_binding_curves()

    def auto_fit_pair(self, pair, rlow=0.35, rhigh=3.5, npoints=24):
        """
        Fit Morse potential parameters (ε, ρ₀, r₀) for a given pair.
        """
        r_vals, E_bind = self.compute_binding_curve(pair, rlow, rhigh, npoints)
        E_bind *= HARTREE_TO_EV

        def morse_energy(r, epsilon, rho0, r0):
            return epsilon * (
                np.exp(-2 * rho0 * (r / r0 - 1)) - 2 * np.exp(-rho0 * (r / r0 - 1))
            )

        min_idx = np.argmin(E_bind)
        init_epsilon = abs(E_bind[min_idx])
        init_r0 = r_vals[min_idx]
        init_rho0 = 2.0
        init_params = [init_epsilon, init_rho0, init_r0]

        def objective(params):
            epsilon, rho0, r0 = params
            return np.mean((morse_energy(r_vals, epsilon, rho0, r0) - E_bind) ** 2)

        res = minimize(
            objective,
            init_params,
            method="L-BFGS-B",
            tol=1.0e-12,
            options=self.opt_kwargs,
        )
        if not res.success:
            raise RuntimeError(f"Optimization failed for pair {pair}: {res.message}")

        fitted_epsilon, fitted_rho0, fitted_r0 = res.x
        params = {"epsilon": fitted_epsilon, "rho0": fitted_rho0, "r0": fitted_r0}

        return params, r_vals, E_bind

    def compute_binding_curve(self, pair, rlow, rhigh, npoints):
        """
        Compute binding curve for a given pair.
        """
        elem1, elem2 = pair
        charge, spin = self._get_charge_spin(pair)
        basis = self.qc_kwargs["basis"]
        method = self.qc_kwargs["method"].upper()

        atom_energies = {}
        for elem in pair:
            iso_spin = self._get_isolated_spin(elem)
            mol = gto.M(atom=elem, basis=basis, charge=0, spin=iso_spin, verbose=0)
            mf = scf.RHF(mol).run() if iso_spin == 0 else scf.UHF(mol).run()
            atom_energies[elem] = self._run_level_of_theory(mf, method)

        distances = np.linspace(rlow, rhigh, npoints)
        E_bind = []
        for r in tqdm(distances, desc=f"Computing binding curve for {pair}"):
            mol = gto.M(
                atom=f"{elem1} 0 0 0; {elem2} 0 0 {r}",
                basis=basis,
                charge=charge,
                spin=spin,
                verbose=0,
            )
            mf = scf.RHF(mol).run() if spin == 0 else scf.UHF(mol).run()
            E_total = self._run_level_of_theory(mf, method)
            binding = E_total - (atom_energies[elem1] + atom_energies[elem2])
            E_bind.append(binding)
        return distances, np.array(E_bind)

    def _save_parameters(self):
        """
        Save Morse parameters to both YAML and TOML formats.
        """

        with open(f"Morse_{self.molecule}.parameters.toml", "w") as toml_file:
            toml.dump(self.pair_params, toml_file)

    def _plot_binding_curves(self):
        """
        Plot all computed binding curves and fitted Morse potential for each series.
        """
        plt.figure(figsize=(8, 6))

        for pair, (r_vals, E_bind) in self.binding_curves.items():
            label = f"{pair[0]}-{pair[1]}"

            params = self.pair_params["-".join(pair)]
            epsilon, rho0, r0 = params["epsilon"], params["rho0"], params["r0"]

            def morse_energy(r, epsilon, rho0, r0):
                return epsilon * (
                    np.exp(-2 * rho0 * (r / r0 - 1)) - 2 * np.exp(-rho0 * (r / r0 - 1))
                )

            fitted_curve = morse_energy(r_vals, epsilon, rho0, r0)

            (line,) = plt.plot(
                r_vals, fitted_curve, linestyle="--", label=f"{label} (Fit)"
            )
            color = line.get_color()

            plt.scatter(
                r_vals,
                E_bind,
                label=f"{label} (Ref)",
                marker="o",
                color=color,
                s=30,
                edgecolors="k",
            )

        method = self.qc_kwargs.get("method", "Unknown Method")
        basis = self.qc_kwargs.get("basis", "Unknown Basis")

        plt.xlabel("Bond length (Å)")
        plt.ylabel("Binding energy (eV)")
        plt.title(f"Binding Curves from PySCF Calculations\n{method} | {basis}")
        plt.legend()
        plt.savefig(f"Morse_{self.molecule}_binding_curves.png")
        plt.close()

    def _get_isolated_spin(self, elem):
        Z = atomic_numbers.get(elem)
        if Z is None:
            raise ValueError(f"Atomic number for element {elem} not found.")
        return 1 if Z % 2 == 1 else 0

    def _get_charge_spin(self, pair):
        total = sum(atomic_numbers.get(elem, 0) for elem in pair)
        return 0, 0 if total % 2 == 0 else 1

    def _run_level_of_theory(self, mf, method):
        if method == "HF":
            return mf.e_tot
        elif method == "MP2":
            return mp.MP2(mf).run().e_tot
        elif method == "CCSD":
            return cc.CCSD(mf).run().e_tot
        elif method == "CCSD(T)":
            return cc.CCSD(mf).run().e_tot
        else:
            raise ValueError(f"Unsupported method: {method}")

In [68]:
from ase.atoms import Atoms
pair="SiN"
qc_kwargs = {'method': 'CCSD', 'basis': 'aug-cc-pVDZ'}
calc = AutoMorsePotential(molecule=pair,qc_kwargs=qc_kwargs)

atoms = Atoms(pair, positions=[[0, 0, 0], [1.5, 0, 0]])
atoms.calc = calc
energy = atoms.get_potential_energy()
forces = atoms.get_forces()

print("Total Energy:", energy)
print("Forces:\n", forces)

Using level of theory: CCSD | Basis set: aug-cc-pVDZ


Computing binding curve for ('Si', 'Si'): 100%|██████████| 24/24 [02:24<00:00,  6.02s/it]


Total Energy: -0.8451818782538245
Forces:
 [[ 0.47730244  0.          0.        ]
 [-0.47730244  0.          0.        ]]
